# Dataset: *charliehebdo*

- df.to_csv(r'D:\論文\PHEME9\Data\CSV\charliehebdo-df.csv',index=False)
- src_tw_df.to_csv(r'D:\論文\PHEME9\Data\CSV\charliehebdo-src.csv',index=False)

In [1]:
import numpy as np
import pandas as pd
import datetime 
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
from transformers import AutoTokenizer, AutoModelWithLMHead, AutoModelForSequenceClassification

INFO:tensorflow:Enabling eager execution
INFO:tensorflow:Enabling v2 tensorshape
INFO:tensorflow:Enabling resource variables
INFO:tensorflow:Enabling tensor equality
INFO:tensorflow:Enabling control flow v2


In [2]:
df = pd.read_csv(r'D:\論文\PHEME9\Data\CSV\charliehebdo-df.csv', encoding = 'utf-8', header = 0)
src_tw_df = pd.read_csv(r'D:\論文\PHEME9\Data\CSV\charliehebdo-src.csv', encoding = 'utf-8', header =0)
print(df.shape)
print(src_tw_df.shape)

(36189, 11)
(2002, 7)


In [3]:
df.loc[23,['reply_tweet','cleaned_reply_tw','reply_sentiment']]

reply_tweet         @neesa_h @GabTarquini @BBCDanielS @BBCWorld Gr...
cleaned_reply_tw         great not see thousands protest hijack islam
reply_sentiment                                              Positive
Name: 23, dtype: object

In [11]:
df.isnull().sum()

src_tweet_id           0
src_user_id            0
src_tweet              0
src_date               0
reply_tweet_id         0
reply_user_id          0
reply_tweet            0
reply_date             0
label                  0
cleaned_reply_tw    2561
reply_sentiment        0
dtype: int64

In [12]:
df.dropna(inplace = True)
df.reset_index(drop=True,inplace=True)
df.shape

(33628, 11)

In [13]:
src_tw_df.isnull().sum()

src_tweet_id       0
src_user_id        0
src_tweet          0
src_date           0
label              0
cleaned_src_tw    17
src_sentiment      0
dtype: int64

In [14]:
src_tw_df[src_tw_df.cleaned_src_tw.isnull()].src_tweet

119     RT #JeSuisCharlie In support of #CharlieHebdo....
267     “If liberty means anything at all, it means th...
328     RT to fuck terrorism http://t.co/7SHjMeJ0iD ht...
344     RT this cartoon and pay your tribute\nshow the...
377     RT this cartoon and pay your tribute\nshow the...
392     RT @froomkin: #CharlieHebdo’s Charb: "I would ...
686     RT @socdec: #Charb, cartoonist killed in #Pari...
753     RT @mkapral Today's French lesson. #JeSuisChar...
765     RT @Beltrew: A few of the best cartoons drawn ...
807     RT @LatuffCartoons: So now let's talk seriousl...
903     “17 journalists massacres by Israel in Gaza, 2...
974     RT @charliesaidthat: The Earth without Art is ...
1084    RT - If you support this #freedom fighter! #St...
1198    THIS: RT @LatuffCartoons: let's talk seriously...
1639    “@LePoint: #CharlieHebdo : “The cartoonists Ch...
1665    “I don’t feel as though I’m killing someone wi...
1716    RT @JeffersonObama: R.I.P. Ahmed Merabet, a Fr...
Name: src_twee

In [15]:
a = src_tw_df[src_tw_df.cleaned_src_tw.isnull()].src_tweet_id.values
df.loc[df.src_tweet_id.isin(a),['reply_tweet','cleaned_reply_tw']]

,reply_tweet,cleaned_reply_tw
2649,@Journalism2ls #jesuischarlie,jesuischarlie
2650,This. And buy a subscription as well ! RT@Jour...,buy subscription well
5394,@LibertarianGeo @mimitonadzuketa I recognise t...,recognise guy right
5395,@BreadunCircuses Orwell,orwell
5396,@LibertarianGeo Thank you! (It was bugging me ...,thank bug couldnt remember
...,...,...
30198,@tonyjdell @Mama4Obama1 @JeffersonObama worldwide,worldwide
30199,@No_Idteaots @Mama4Obama1 @JeffersonObama Yea...,yeah chop head aint chop theyre cheer
30200,.@tonyjdell @No_Idteaots @JeffersonObama thank...,thank make easy spot block bye
30201,@tonyjdell @No_Idteaots @JeffersonObama Seriou...,seriously theyve give chance reform idiotic bu...


## Emotion Analysis

### T5 Based
<https://huggingface.co/mrm8488/t5-base-finetuned-emotion?text=I+wish+you+were+here+but+it+is+impossible>

In [ ]:
t5_tokenizer = AutoTokenizer.from_pretrained("mrm8488/t5-base-finetuned-emotion",use_fast=False)
t5_model = AutoModelWithLMHead.from_pretrained("mrm8488/t5-base-finetuned-emotion")



def get_emotion_t5(text):
    input_ids = t5_tokenizer.encode(text , return_tensors='pt')
    output = t5_model.generate(input_ids=input_ids,max_length=2)
  
    dec = [t5_tokenizer.decode(ids) for ids in output]
    label = dec[0]
    return label

In [ ]:
emotion_t5 = df.cleaned_reply_tw.astype(str).apply(get_emotion_t5)

In [ ]:
df['reply_emo_t5'] = emotion_t5

In [ ]:
import re
df['reply_emo_t5'] = df['reply_emo_t5'].apply(lambda x : re.sub('<pad> ',"",x))

In [ ]:
src_emo_t5 = src_tw_df.cleaned_src_tw.astype(str).apply(get_emotion_t5)
src_tw_df['src_emo_t5'] = src_emo_t5

In [ ]:
src_tw_df['src_emo_t5']  = src_tw_df.src_emo_t5.apply(lambda x : re.sub('<pad> ',"",x))

### roBERTa-base model
<https://huggingface.co/cardiffnlp/twitter-roberta-base-emotion>

In [5]:
import numpy as np
from scipy.special import softmax

task='emotion'
MODEL = f"cardiffnlp/twitter-roberta-base-{task}"
ro_tokenizer = AutoTokenizer.from_pretrained(MODEL)
ro_model = AutoModelForSequenceClassification.from_pretrained(MODEL)


In [6]:
def get_emotion_roBERT(text):
    
    def preprocess(txt):
        new_text = []
        for t in txt.split(" "):
            t = '@user' if t.startswith('@') and len(t) > 1 else t
            t = 'http' if t.startswith('http') else t
            new_text.append(t)
        return " ".join(new_text)
    
    labels = ['anger','joy','optimism','sadness']
    text = preprocess(text)
    encoded_input = ro_tokenizer(text, return_tensors='pt')
    output = ro_model(**encoded_input)

    scores = output[0][0].detach().numpy()
    scores = softmax(scores)
    ranking = np.argsort(scores)
    ranking = ranking[::-1]
    emo = labels[ranking[0]]
    return emo

In [7]:
emotion_ro = df.cleaned_reply_tw.apply(get_emotion_roBERT)

In [8]:
df['reply_emo_ro'] = emotion_ro

In [10]:
df.to_csv(r'D:\論文\PHEME9\Data\CSV\charliehebdo-df-2.csv',index=False)

In [11]:
src_emo_ro = src_tw_df.cleaned_src_tw.astype(str).apply(get_emotion_roBERT)
src_tw_df['src_emo_ro'] = src_emo_ro

In [12]:
src_tw_df.to_csv(r'D:\論文\PHEME9\Data\CSV\charliehebdo-src-2.csv',index=False)

 - robert : 'anger','joy','optimism','sadness'
 - t5: 'sadness', 'joy', 'love', 'anger', 'fear', 'surprise' 
 - text2emotion: 'angry, 'fear', 'happy, 'sad', 'surprise'

In [ ]:
print("source tweet:", df.src_tweet[1118],'\n')
print("original reply tw: ", df.reply_tweet[1118])
print("cleaned reply tw: ", df.cleaned_reply_tw[1118])

print('roBERT based: ',df.reply_emo_ro[1118])
print('t5 based: ',df.reply_emo_t5[1118])
print('text2emotion: ', df.reply_emo_te[1118])